# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 9:27AM,237819,10,CLI00280.1,"CLINUVEL, Inc.",Released
1,May 23 2022 9:27AM,237819,10,CLI00281.1,"CLINUVEL, Inc.",Released
2,May 23 2022 9:21AM,237818,10,CLI00276.1,"CLINUVEL, Inc.",Released
3,May 23 2022 9:21AM,237818,10,CLI00277.1,"CLINUVEL, Inc.",Released
4,May 23 2022 9:21AM,237818,10,CLI00278.1,"CLINUVEL, Inc.",Released
5,May 23 2022 9:20AM,237814,10,0085694475,ISDIN Corporation,Released
6,May 23 2022 9:20AM,237814,10,0085694808,ISDIN Corporation,Released
7,May 23 2022 9:20AM,237814,10,0085695068,ISDIN Corporation,Released
8,May 23 2022 9:20AM,237814,10,0085695069,ISDIN Corporation,Released
9,May 23 2022 9:20AM,237814,10,0085695076,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,237815,Released,1
26,237816,Released,1
27,237817,Released,3
28,237818,Released,3
29,237819,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237815,NaN,NaN,1.0
237816,NaN,NaN,1.0
237817,NaN,NaN,3.0
237818,NaN,NaN,3.0
237819,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1.0,16.0,17.0
237781,0.0,0.0,51.0
237787,0.0,0.0,1.0
237789,0.0,6.0,1.0
237792,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1,16,17
237781,0,0,51
237787,0,0,1
237789,0,6,1
237792,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,16,17
1,237781,0,0,51
2,237787,0,0,1
3,237789,0,6,1
4,237792,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,16,17
1,237781,,,51
2,237787,,,1
3,237789,,6,1
4,237792,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc."
2,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc."
5,May 23 2022 9:20AM,237814,10,ISDIN Corporation
33,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC"
36,May 23 2022 9:17AM,237812,10,ISDIN Corporation
86,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc."
87,May 23 2022 9:16AM,237815,18,"ClearSpec, LLC"
88,May 23 2022 9:13AM,237810,10,ISDIN Corporation
134,May 23 2022 9:13AM,237813,10,"Amcyte Pharma, Inc."
137,May 23 2022 9:08AM,237807,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",,,2
1,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",,,3
2,May 23 2022 9:20AM,237814,10,ISDIN Corporation,,,28
3,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",,,3
4,May 23 2022 9:17AM,237812,10,ISDIN Corporation,,,50
5,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc.",,,1
6,May 23 2022 9:16AM,237815,18,"ClearSpec, LLC",,,1
7,May 23 2022 9:13AM,237810,10,ISDIN Corporation,,,46
8,May 23 2022 9:13AM,237813,10,"Amcyte Pharma, Inc.",,,3
9,May 23 2022 9:08AM,237807,15,"Virtus Pharmaceuticals, LLC",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,,
1,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",3,,
2,May 23 2022 9:20AM,237814,10,ISDIN Corporation,28,,
3,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",3,,
4,May 23 2022 9:17AM,237812,10,ISDIN Corporation,50,,
5,May 23 2022 9:17AM,237816,21,"NBTY Global, Inc.",1,,
6,May 23 2022 9:16AM,237815,18,"ClearSpec, LLC",1,,
7,May 23 2022 9:13AM,237810,10,ISDIN Corporation,46,,
8,May 23 2022 9:13AM,237813,10,"Amcyte Pharma, Inc.",3,,
9,May 23 2022 9:08AM,237807,15,"Virtus Pharmaceuticals, LLC",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,,
1,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",3,,
2,May 23 2022 9:20AM,237814,10,ISDIN Corporation,28,,
3,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",3,,
4,May 23 2022 9:17AM,237812,10,ISDIN Corporation,50,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,NaN,NaN
1,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",3,NaN,NaN
2,May 23 2022 9:20AM,237814,10,ISDIN Corporation,28,NaN,NaN
3,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",3,NaN,NaN
4,May 23 2022 9:17AM,237812,10,ISDIN Corporation,50,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:27AM,237819,10,"CLINUVEL, Inc.",2,0.0,0.0
1,May 23 2022 9:21AM,237818,10,"CLINUVEL, Inc.",3,0.0,0.0
2,May 23 2022 9:20AM,237814,10,ISDIN Corporation,28,0.0,0.0
3,May 23 2022 9:18AM,237817,10,"Nextsource Biotechnology, LLC",3,0.0,0.0
4,May 23 2022 9:17AM,237812,10,ISDIN Corporation,50,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2615867,236,16.0,1.0
15,1426797,80,0.0,0.0
18,1664611,50,0.0,0.0
19,237789,1,6.0,0.0
21,237816,1,0.0,0.0
22,237787,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2615867,236,16.0,1.0
1,15,1426797,80,0.0,0.0
2,18,1664611,50,0.0,0.0
3,19,237789,1,6.0,0.0
4,21,237816,1,0.0,0.0
5,22,237787,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,236,16.0,1.0
1,15,80,0.0,0.0
2,18,50,0.0,0.0
3,19,1,6.0,0.0
4,21,1,0.0,0.0
5,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,236.0
1,15,Released,80.0
2,18,Released,50.0
3,19,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,0.0,6.0,0.0,0.0
Released,236.0,80.0,50.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,6.0,0.0,0.0
2,Released,236.0,80.0,50.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,6.0,0.0,0.0
2,Released,236.0,80.0,50.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()